In [1]:
from laptoppriceworkflow.data import get_json_data 
from laptoppriceworkflow.functions import clean_col_meta
from laptoppriceworkflow.functions import simplify_gwarancja
from laptoppriceworkflow.functions import simplify_komunikacja

import pandas as pd

import numpy as np

import seaborn as sns 

import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline, make_union

from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

In [3]:
df = get_json_data()

In [4]:
msk = np.random.rand(len(df)) < 0.75
train = df[msk]
test = df[~msk]

In [5]:
train.head(2)

,buynow_price,rodzaj karty graficznej,komunikacja,rozdzielczość (piksele),liczba rdzeni procesora,wielkość pamięci ram,system operacyjny,typ dysku twardego,sterowanie,multimedia,typ pamięci ram,taktowanie bazowe procesora (ghz),seria procesora,stan,pojemność dysku (gb),gwarancja,wielkość matrycy
70,1599.0,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,8 gb,[windows 10 home],hdd,"[klawiatura, touchpad]","[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr3,1.9,intel core i3,nowy,500.0,sprzedawcy,"13"" - 13.9"""
1489,2999.0,grafika dedykowana,"[nfc (near field communication), gps]",1920 x 1080,2,12 gb,[windows 10 home],None,[touchpad],"[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr4,2.0,intel core i3,nowy,480.0,producenta,"15"" - 15.9"""


In [ ]:
train_num = train[['taktowanie bazowe procesora (ghz)', 'pojemność dysku (gb)', 'wielkość pamięci ram']]

In [35]:
class FeatureEng(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        #imm = joblib.load(PATH_TO_IMM)
        #metadata_dense = reduce_mem_usage(pd.read_csv(METADATA_DENSE).fillna(0))
        #X["country"] = X["city"].map(lambda x: x.split(",")[-1].strip())
        X["rodzaj karty graficznej"].fillna("", inplace=True)
        X["komunikacja"].fillna("", inplace=True)
        X["rozdzielczość (piksele)"].fillna("", inplace=True)
        
        X['rozdzielczosc_piksele'] = X["rozdzielczość (piksele)"].map(lambda x: 1 if x == '1920 x 1080' else 0)
        X.drop('rozdzielczość (piksele)', axis = 1)
        
        #X['liczba_rdzeni_proc_trans'] = ['2' if x == '2' else '' for x in X.loc[:,"liczba rdzeni procesora"]]
        X['liczba_rdzeni_proc_trans'] = X['liczba rdzeni procesora'].map(lambda x: '2' if x == '2' else '')
        
        X.drop('liczba rdzeni procesora', axis = 1)
        
        X["wielkość pamięci ram"].fillna('0', inplace=True)
        
        #X["wielkosc_pamieci_ram_trans"] = [0 if "mb" in x else int(x.strip('gb')) for x in X.loc[:, "wielkość pamięci ram"]]
        X["wielkosc_pamieci_ram_trans"] = X["wielkość pamięci ram"].map(lambda x: 0 if "mb" in x else int(x.strip('gb')))
        
        X.drop("wielkość pamięci ram", axis = 1)
        
        X["system operacyjny"].fillna('', inplace=True)
        X["typ dysku twardego"].fillna('', inplace=True)
        X["lan"] = X['komunikacja'].map(lambda x: 1 if any("lan" in i for i in x) else 0)
        X["wi-fi"] = X['komunikacja'].map(lambda x: 1 if any("wi-fi" in i for i in x) else 0)
        X["nfc"] = X['komunikacja'].map(lambda x: 1 if any("nfc" in i for i in x) else 0)
        
        X.drop("komunikacja", axis = 1)
        
        X['windows'] = X['system operacyjny'].map(lambda x: 1 if any("windows" in i for i in x) else 0)
        X.drop("system operacyjny", axis = 1)
        
        return X

In [21]:
vectorizer = FeatureEng()
ddd = vectorizer.fit_transform(test)
ddd
#ddd  = FeatureEng.fit_transform(train)

C:\Users\marcin\Miniconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\marcin\Miniconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\marcin\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,buynow_price,rodzaj karty graficznej,komunikacja,rozdzielczość (piksele),liczba rdzeni procesora,wielkość pamięci ram,system operacyjny,typ dysku twardego,sterowanie,multimedia,typ pamięci ram,taktowanie bazowe procesora (ghz),seria procesora,stan,pojemność dysku (gb),gwarancja,wielkość matrycy,liczba_rdzeni_proc_trans,wielkosc_pamieci_ram_trans,lan,wi-fi,nfc,windows
4586,1769.00,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,8 gb,[windows 10 home],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr3,1.7,intel core i3,nowy,256.0,sprzedawcy,"15"" - 15.9""",2,8,1,1,0,1
7524,5043.00,grafika dedykowana,"[bluetooth, lan 10/100 mbps]",1366 x 768,2,8 gb,[windows 10 home],ssd,"[klawiatura, touchpad]","[czytnik kart pamięci, kamera, głośniki]",ddr3,2.6,intel core i5,nowy,1000.0,producenta,"15"" - 15.9""",2,8,1,0,0,1
1817,4299.00,grafika dedykowana,,1920 x 1080,4,0,,ssd + hdd,None,None,None,NaN,intel core i5,nowy,NaN,producenta,"15"" - 15.9""",,0,0,0,0,0
5561,899.00,,,,2,0,,,None,None,None,NaN,None,nowy,NaN,producenta,None,2,0,0,0,0,0
5801,1877.00,grafika zintegrowana,"[bluetooth, nfc (near field communication), modem 3g (wwan)]",1600 x 900,4,8 gb,[windows 10 home],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr3l,2.0,amd a6,nowy,128.0,producenta,"17"" - 17.9""",,8,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11455,4849.00,grafika dedykowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1920 x 1080,4,16 gb,[windows 10 home],ssd + hdd,"[klawiatura, touchpad, klawiatura podświetlana, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr4,2.8,intel core i7,nowy,1250.0,producenta,"15"" - 15.9""",,16,1,1,0,1
6115,4296.55,grafika dedykowana,[bluetooth],1920 x 1080,2,8 gb,[windows 10 professional],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr3,2.4,intel core i7,nowy,256.0,sprzedawcy,"15"" - 15.9""",2,8,0,0,0,1
6423,4599.00,grafika dedykowana,"[bluetooth, lan 10/100/1000 mbps]",1920 x 1080,4,32 gb,[brak systemu],hdd,"[klawiatura, touchpad, klawiatura podświetlana, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr4,2.6,intel core i7,nowy,1000.0,producenta,"17"" - 17.9""",,32,1,0,0,0
10541,1329.00,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,4 gb,[windows 10 home],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr3,1.6,intel celeron dual-core,nowy,128.0,sprzedawcy,"15"" - 15.9""",2,4,1,1,0,1


In [7]:
class PandasToNpArray(BaseEstimator, TransformerMixin):
    def fit(self, X, *arg):
        return self

    def transform(self, X):
        return X.values.astype(np.float)

In [36]:
preprocessor = make_pipeline(
        FeatureEng(),
            ColumnTransformer(
            [
                (
                    "cat_vars",
                    make_pipeline(OneHotEncoder(sparse=False)),
                    #list(['gwarancja']),
                    ['rodzaj karty graficznej','gwarancja','rozdzielczosc_piksele','liczba_rdzeni_proc_trans'],
                ),
               (
                    "ram",
                    make_pipeline(PandasToNpArray(), SimpleImputer(strategy="constant", fill_value=-9999)),
                    ['wielkosc_pamieci_ram_trans'],
                ),
              (
                    "other_vars",
                    make_pipeline(PandasToNpArray()),
                    ['lan','wi-fi', 'nfc', 'windows','rozdzielczosc_piksele'],
                ),
                
                
                
            ]
        )
)

In [23]:
preprocessor.fit_transform(test)

C:\Users\marcin\Miniconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\marcin\Miniconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\marcin\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

array([[0., 0., 1., ..., 1., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 1., 0., ..., 0., 1., 1.]])

In [48]:
from sklearn.linear_model import Ridge

In [49]:
ridge_model = Ridge(
    solver='auto', fit_intercept=True, alpha=1.0,
    max_iter=100, normalize=False, tol=0.05, random_state = 1,
)

In [52]:
pipeline = make_pipeline(
    preprocessor,
    ridge_model
)

In [71]:
pipeline.fit(train, train['buynow_price'])
preds = pipeline.predict(test)

lin_mse = mean_squared_error(preds, test['buynow_price'])
lin_rmse = np.sqrt(lin_mse)
lin_rmse

C:\Users\marcin\Miniconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\marcin\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\marcin\Miniconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

1221.2939213313769

In [70]:
preds

array([1689.67788973, 3108.32542905, 3952.942178  , ..., 6468.26184174,
       1336.47929431, 5156.50532839])